This lab guides learners through the process of extracting data from a RESTful API using Python's requests library and integrating it into a MySQL database. Learners will fetch JSON data from a given API endpoint, establish a connection to a MySQL database, create a table, and insert the fetched data into the table.

Main Execution:

The main part of the code fetches data from the API using fetch_posts(), loads it into the MySQL database using load_to_mysql(data), and then fetches and displays the data from the MySQL database using fetch_from_mysql()

In [ ]:
# Importing the requests module to make HTTP requests (for API)
import requests

# Importing the MySQL connector module and giving it an alias
import mysql.connector as dbconnect

# ------------------- FUNCTION 1: Fetch posts from API -------------------

# This function fetches posts data from a public API endpoint
def fetch_posts():
    # Define the URL of the API we want to get data from
    url = 'https://jsonplaceholder.typicode.com/posts/'

    # Send a GET request to the API
    response = requests.get(url)

    # If the request was successful (HTTP 200), return the data as JSON
    if response.status_code == 200:
        return response.json()
    else:
        # Otherwise, print the error status and return None
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

# ------------------- FUNCTION 2: Load fetched data into MySQL -------------------

# This function loads data into a MySQL table
def load_to_mysql(data):
    try:
        # Connect to the MySQL database using provided credentials
        conn = dbconnect.connect(
            host='localhost',       # MySQL server address
            port='3306',            # Port where MySQL is running (3306 is default)
            user='root',            # MySQL username
            password='password',    # MySQL password
            database='classicmodels'  # Target database name
        )

        # Create a cursor object which is used to execute SQL queries
        cursor = conn.cursor()

        # Create a table named 'posts' if it doesn't already exist
        # The table will have columns matching the API structure
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS posts (
                id INT PRIMARY KEY,
                userId INT,
                title VARCHAR(255),
                body TEXT
            )
        ''')

        # Loop through each post in the data and insert it into the 'posts' table
        for post in data:
            cursor.execute('''
                INSERT INTO posts (id, userId, title, body)
                VALUES (%s, %s, %s, %s)
            ''', (post['id'], post['userId'], post['title'], post['body']))

        # Commit the changes to make them permanent in the database
        conn.commit()

        # Close the cursor and connection to free up resources
        cursor.close()
        conn.close()

        print("Data loaded successfully to MySQL database.")

    except Exception as e:
        # Print the error message if something goes wrong
        print(f"Error: {e}")

# ------------------- FUNCTION 3: Fetch and display data from MySQL -------------------

# This function reads and prints the data from the MySQL 'posts' table
def fetch_from_mysql():
    try:
        # Connect to the MySQL database again to read data
        myconnection = dbconnect.connect(
            host='localhost',
            database='classicmodels',
            user='root',
            password='password',
            port='3306'
        )

        # Check if the connection is active
        if myconnection.is_connected():
            print('Successfully Connected to MySQL database')

            # Create a cursor to execute the SELECT query
            cursor = myconnection.cursor()

            # Define the SQL query to select all records from 'posts' table
            SQLQuery = "SELECT * FROM posts"

            # Execute the query
            cursor.execute(SQLQuery)

            # Fetch all results from the executed query
            records = cursor.fetchall()

            # Print number of records found
            print("Total number of rows in table: ", cursor.rowcount)

            print("\nPrinting each row")

            # Loop through the records and print each one
            for row in records:
                print("Post ID       = ", row[0])
                print("User ID       = ", row[1])
                print("Post Title    = ", row[2])
                print("Post Body     = ", row[3])
                print("-------------------------------------")

    # Catching any MySQL-related errors
    except dbconnect.Error as e:
        print("Error while connecting to Database", e)

    # Ensure the connection is closed even if there is an error
    finally:
        if myconnection.is_connected():
            cursor.close()
            myconnection.close()
            print("Database connection is closed")

# ------------------- MAIN EXECUTION STARTS HERE -------------------

# Call the function to fetch posts from the API
posts_data = fetch_posts()

# If the API call was successful and we have data
if posts_data:
    # Load the fetched data into the MySQL database
    load_to_mysql(posts_data)

    # Read and print the data from the MySQL database
    fetch_from_mysql()
